#### Movielens - merge datasets into one

In [1]:
import pandas as pd

# ml-25m datasets
ml_movie = pd.read_csv('ml-25m/movies.csv')
link = pd.read_csv('ml-25m/links.csv')

In [2]:
# use pyspark to calculate the average ratings for each movie
from pyspark.sql.functions import col
import pyspark.sql.functions as func

#read ratings.csv
df = spark.read.option('header','true')\
               .option('inferSchema','true')\
               .csv("ml-25m/ratings.csv")

#group the ratings by movieId and avg()
avg_rating_cal = df.groupBy('movieId')\
                   .agg({'rating':'avg'})\
                   .select(col('movieId'),(col('avg(rating)')).alias('average_rating'))\
                   .orderBy('movieId')

#round up the average rating of each movie
avg_rating = avg_rating_cal.withColumn("average_rating", func.round(avg_rating_cal["average_rating"], 2))
# save output to csv
avg_rating.toPandas().to_csv('ml_avg_rating.csv')

In [3]:
# merge movielens' datasets
avg_ratings = pd.read_csv('ml_avg_rating.csv',index_col=0)
df = ml_movie.merge(avg_ratings, on='movieId')

In [4]:
df

,movieId,title,genres,average_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.89
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.25
2,3,Grumpier Old Men (1995),Comedy|Romance,3.14
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.85
4,5,Father of the Bride Part II (1995),Comedy,3.06
...,...,...,...,...
59042,209157,We (2018),Drama,1.50
59043,209159,Window of the Soul (2001),Documentary,3.00
59044,209163,Bad Poems (2018),Comedy|Drama,4.50
59045,209169,A Girl Thing (2001),(no genres listed),3.00


In [5]:
df.to_csv('movielens.csv')

#### IMDb - merge datasets into one

In [2]:
import pandas as pd
#imdb datasets
imdb_info = pd.read_csv('IMDb/title.basics.tsv', sep='\t')
imdb_crew = pd.read_csv('IMDb/title.crew.tsv', sep='\t')
imdb_principals = pd.read_csv('IMDb/title.principals.tsv', sep='\t')
imdb_ratings = pd.read_csv('IMDb/title.ratings.tsv', sep='\t')
imdb_name = pd.read_csv('IMDb/name.basics.tsv', sep='\t')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
imdb_info['titleType'].value_counts()

tvEpisode       4780697
short            734305
movie            548388
video            261363
tvSeries         182117
tvMovie          120559
tvMiniSeries      30366
tvSpecial         28807
videoGame         25316
tvShort           12482
Name: titleType, dtype: int64

In [34]:
# find and remove irrelevent type, only movie
not_rel = ['short','video','tvEpisode','tvSeries','tvMovie','tvMiniSeries','tvSpecial','videoGame','tvShort']
index_list = []
for i,value in enumerate(imdb_info['titleType']):
    if value in not_rel:
        index_list.append(i)
        
df2 = imdb_info.drop(index=index_list)
df2 = df2.reset_index()
df2 = df2.drop(columns=['index','originalTitle','endYear'])
df2['titleType'].value_counts()

movie    548388
Name: titleType, dtype: int64

In [35]:
df2.head(10)

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0000009,movie,Miss Jerry,0,1894,45,Romance
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,0,1897,20,"Documentary,News,Sport"
2,tt0000335,movie,Soldiers of the Cross,0,1900,\N,"Biography,Drama"
3,tt0000502,movie,Bohemios,0,1905,100,\N
4,tt0000574,movie,The Story of the Kelly Gang,0,1906,70,"Biography,Crime,Drama"
5,tt0000615,movie,Robbery Under Arms,0,1907,\N,Drama
6,tt0000630,movie,Hamlet,0,1908,\N,Drama
7,tt0000675,movie,Don Quijote,0,1908,\N,Drama
8,tt0000676,movie,Don Álvaro o la fuerza del sino,0,1908,\N,Drama
9,tt0000679,movie,The Fairylogue and Radio-Plays,0,1908,120,"Adventure,Fantasy"


In [36]:
df2 = df2.merge(imdb_crew, on='tconst')
df2 = df2.merge(imdb_ratings, on='tconst')

In [43]:
df2=pd.DataFrame(df2,columns=['tconst','primaryTitle','startYear','runtimeMinutes','genres','directors','writers','actors','isAdult','averageRating','numVotes'])

In [44]:
df2

,tconst,primaryTitle,startYear,runtimeMinutes,genres,directors,writers,actors,isAdult,averageRating,numVotes
0,tt0000009,Miss Jerry,1894,45,Romance,nm0085156,nm0085156,NaN,0,5.4,108
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",nm0714557,\N,NaN,0,5.2,341
2,tt0000335,Soldiers of the Cross,1900,\N,"Biography,Drama","nm0095714,nm0675140",\N,NaN,0,6.1,40
3,tt0000502,Bohemios,1905,100,\N,nm0063413,"nm0063413,nm0657268,nm0675388",NaN,0,4.4,5
4,tt0000574,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",nm0846879,nm0846879,NaN,0,6.1,567
...,...,...,...,...,...,...,...,...,...,...,...
247038,tt9914942,La vida sense la Sara Amat,2019,74,Drama,nm1716653,"nm3678448,nm9361716",NaN,0,6.7,77
247039,tt9915790,Bobbyr Bondhura,2019,\N,Family,nm10538030,\N,NaN,0,7.6,14
247040,tt9916160,Drømmeland,2019,72,Documentary,nm5684093,\N,NaN,0,6.6,36
247041,tt9916428,The Secret of China,2019,\N,"Adventure,History,War",nm0910951,\N,NaN,0,3.3,11


In [47]:
df2.shape

(247043, 11)

In [39]:
# create actor dict {tconst:actor}
actor_dict ={}
for index, value in enumerate(imdb_principals['category']):
    if value == 'actor':
        actor = imdb_principals['nconst'][index]
        movie = imdb_principals['tconst'][index]
        if movie in actor_dict:
            actor_dict[movie] += ',' + actor
        else:
            actor_dict[movie] = actor

In [48]:
#fill in the df['actors']
for i,value in enumerate(df2['tconst']):
    if value in actor_dict:
        df2['actors'][i] = actor_dict[value]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [49]:
df2

,tconst,primaryTitle,startYear,runtimeMinutes,genres,directors,writers,actors,isAdult,averageRating,numVotes
0,tt0000009,Miss Jerry,1894,45,Romance,nm0085156,nm0085156,"nm0183823,nm1309758",0,5.4,108
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",nm0714557,\N,NaN,0,5.2,341
2,tt0000335,Soldiers of the Cross,1900,\N,"Biography,Drama","nm0095714,nm0675140",\N,"nm1012612,nm1011210,nm1012621,nm0675239,nm0675260",0,6.1,40
3,tt0000502,Bohemios,1905,100,\N,nm0063413,"nm0063413,nm0657268,nm0675388","nm0215752,nm0252720",0,4.4,5
4,tt0000574,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",nm0846879,nm0846879,"nm0846894,nm3002376",0,6.1,567
...,...,...,...,...,...,...,...,...,...,...,...
247038,tt9914942,La vida sense la Sara Amat,2019,74,Drama,nm1716653,"nm3678448,nm9361716","nm10823679,nm1290838,nm5580678",0,6.7,77
247039,tt9915790,Bobbyr Bondhura,2019,\N,Family,nm10538030,\N,"nm1387306,nm10538031,nm3056725,nm1910815",0,7.6,14
247040,tt9916160,Drømmeland,2019,72,Documentary,nm5684093,\N,NaN,0,6.6,36
247041,tt9916428,The Secret of China,2019,\N,"Adventure,History,War",nm0910951,\N,"nm3611859,nm9445072,nm8594703,nm8680851,nm2631...",0,3.3,11


In [50]:
df2.to_csv('imdb.csv')